In [1]:
import pandas as pd
import numpy as np
import glob
import warnings
warnings.filterwarnings('ignore')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_colwidth', -1)  #Veo hasta la última palabra
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [ ]:
'C:/Users/SanchezEdg/MetLife/Perez, Juan Ignacio - COVID19/Data/Regional/'
'C:\Users\SanchezEdg\OneDrive - MetLife\Python Scripts\2020\Covid\COVID-19\csse_covid_19_data\csse_covid_19_daily_reports/'

In [6]:
def generate_Mex_region_report():          
    
    def replace_tildes(series):
            '''
            Regiones sin tildes y en mayúsculas
            '''
            series = series.str.strip()
            series = series.str.replace('á', 'a')
            series = series.str.replace('é', 'e')
            series = series.str.replace('í', 'i')
            series = series.str.replace('ó', 'o')
            series = series.str.replace('ú', 'u')

            series = series = series.str.upper()
            return series
    
    path_home = r"C:\Users\brian\MetLife\Hopkins_region/"
    path_to_files = r'C:\Users\brian\MetLife\Hopkins_region\files' # use your path
#     path_home = r'C:/Users/SanchezEdg/MetLife/Perez, Juan Ignacio - COVID19/Data/Regional/'
#     path_to_files = r'C:\Users\SanchezEdg\OneDrive - MetLife\Python Scripts\2020\Covid\COVID-19\csse_covid_19_data\csse_covid_19_daily_reports/'


    all_files = glob.glob(path_to_files + "/*.csv")

    li = []

    for filename in all_files:
    #     print(filename)
        df = pd.read_csv(filename)
        if 'Last_Update' in df.columns:
            df['Last_Update'] =  pd.to_datetime(df['Last_Update'])
        elif 'Last Update' in df.columns:
            df['Last Update'] =  pd.to_datetime(df['Last Update'])

        li.append(df)
        
    # Concatenado de todos los reportes diarios en un solo DF
    df_daily = pd.concat(li, axis=0, ignore_index=True)
    
    # Limpieza
    df_daily.loc[pd.isnull(df_daily['Country_Region']),'Country_Region'] = df_daily['Country/Region']
    df_daily.loc[pd.isnull(df_daily['Province_State']),'Province_State'] = df_daily['Province/State']
    df_daily.loc[pd.isnull(df_daily['Last_Update']),'Last_Update'] = df_daily['Last Update']
    df_daily.loc[pd.isnull(df_daily['Lat']),'Lat'] = df_daily['Latitude']
    df_daily.loc[pd.isnull(df_daily['Long_']),'Long_'] = df_daily['Longitude']

    df_daily.drop(columns=['Province/State','Last Update','Latitude','Longitude','FIPS','Admin2','Country/Region','Combined_Key'], inplace=True)
    df_daily.sort_values('Last_Update', inplace=True)

    df_daily.columns = ['CONFIRMED_HM','DEATHS_HM','RECOVERED_HM','STATE','COUNTRY_REGION','LAST_UPDATE','Lat','Long','ACTIVE_HM', 'Incidence_Rate', 'Case-fatality_Ratio']
    df_daily['DATE'] = df_daily['LAST_UPDATE'].dt.date
    df_daily['DATE'] =  pd.to_datetime(df_daily['DATE'])
    
    
    # Filtro por mexico y desde que hay datos por región
    mex = df_daily[df_daily['COUNTRY_REGION'] == 'Mexico']
    first_region_data_date = mex.loc[~pd.isnull(mex['STATE']),'DATE'].min()
    mex_region_data = mex[mex['DATE'] >= first_region_data_date]
    
    # Agrego las filas TOTAL de cada día.
    totals_mex_region_data = mex_region_data.groupby(['DATE']).agg({'DEATHS_HM': 'sum',
                            'ACTIVE_HM': 'sum',
                            'RECOVERED_HM': 'sum',
                            'CONFIRMED_HM': 'sum'}).reset_index()
    totals_mex_region_data['STATE'] = 'TOTAL'
    totals_mex_region_data['COUNTRY_REGION'] = 'Mexico'
    mex_region_data = mex_region_data.append(totals_mex_region_data).sort_values(by=['DATE', 'CONFIRMED_HM']).reset_index(drop=True)
    
    
    # Obtengo las diferencias por día de los campos deseados por región.
    mex_region_data['Fallecidos_corr'] = mex_region_data.groupby('STATE')['DEATHS_HM'].transform(lambda x: x.shift())
    mex_region_data['Activo_corr'] = mex_region_data.groupby('STATE')['ACTIVE_HM'].transform(lambda x: x.shift())
    mex_region_data['Recuperado_corr'] = mex_region_data.groupby('STATE')['RECOVERED_HM'].transform(lambda x: x.shift())
    mex_region_data['Confirmados_corr'] = mex_region_data.groupby('STATE')['CONFIRMED_HM'].transform(lambda x: x.shift())

    mex_region_data['DELTA_CONFIRMED_HM'] = mex_region_data['CONFIRMED_HM'] - mex_region_data['Confirmados_corr']
    mex_region_data['DELTA_DEATHS_HM'] = mex_region_data['DEATHS_HM'] - mex_region_data['Fallecidos_corr']
    mex_region_data['DELTA_RECOVERED_HM'] = mex_region_data['RECOVERED_HM'] - mex_region_data['Recuperado_corr']
    mex_region_data['DELTA_ACTIVE_HM'] = mex_region_data['ACTIVE_HM'] - mex_region_data['Activo_corr']
    
    
    # Limpieza
    mex_region_data = mex_region_data.drop(['Lat', 'Long', 'Incidence_Rate', 'Case-fatality_Ratio'], axis=1)
    mex_region_data = mex_region_data.drop(['Fallecidos_corr', 'Activo_corr', 'Recuperado_corr', 'Confirmados_corr'], axis=1)
    
    mex_region_data = mex_region_data.reindex(['COUNTRY_REGION', 'STATE', 'DATE', 'CONFIRMED_HM', 'DEATHS_HM', 'RECOVERED_HM',
            'ACTIVE_HM', 'DELTA_CONFIRMED_HM', 'DELTA_DEATHS_HM', 'DELTA_RECOVERED_HM', 'DELTA_ACTIVE_HM'], axis=1)

    # Agrego columnas para que coincida exactamente al formato de tabla de regiones_ALL
    mex_region_data['UCI_HM'] = np.NaN
    mex_region_data['DELTA_UCI_HM'] = np.NaN
    mex_region_data['TESTS_HM'] = np.NaN
    mex_region_data['DELTA_TESTS_HM'] = np.NaN
    mex_region_data['LAST_UPDATE'] = mex_region_data['DATE'].max()
    
    # Ajusto los nombres de las regiones para que coincidan con regiones_ALL
    mex_region_data['STATE'] = replace_tildes(mex_region_data['STATE'])
    mex_region_data['STATE'] = mex_region_data['STATE'].str.replace("COAHUILA", "COAHUILA DE ZARAGOZA")
    mex_region_data['STATE'] = mex_region_data['STATE'].str.replace("MICHOACAN", "MICHOACAN DE OCAMPO")
    mex_region_data['STATE'] = mex_region_data['STATE'].str.replace("VERACRUZ", "VERACRUZ DE IGNACIO DE LA LLAVE")
    
    
    mex_region_data.to_csv(path_home + 'reporte_Mexico_region_hopkins.csv', sep='|', encoding='UTF-8', index=False)
    
    return mex_region_data

In [7]:
generate_Mex_region_report()

,COUNTRY_REGION,STATE,DATE,CONFIRMED_HM,DEATHS_HM,RECOVERED_HM,ACTIVE_HM,DELTA_CONFIRMED_HM,DELTA_DEATHS_HM,DELTA_RECOVERED_HM,DELTA_ACTIVE_HM,UCI_HM,DELTA_UCI_HM,TESTS_HM,DELTA_TESTS_HM,LAST_UPDATE
0,Mexico,COLIMA,2020-05-21,83.0,10.0,39.0,34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-09
1,Mexico,DURANGO,2020-05-21,189.0,20.0,116.0,53.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-09
2,Mexico,ZACATECAS,2020-05-21,212.0,27.0,134.0,51.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-09
3,Mexico,CAMPECHE,2020-05-21,328.0,48.0,205.0,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-09
4,Mexico,NAYARIT,2020-05-21,349.0,37.0,207.0,105.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,Mexico,TABASCO,2020-06-09,5667.0,665.0,4004.0,998.0,193.0,15.0,211.0,-33.0,NaN,NaN,NaN,NaN,2020-06-09
656,Mexico,BAJA CALIFORNIA,2020-06-09,6138.0,1257.0,4322.0,559.0,51.0,12.0,92.0,-53.0,NaN,NaN,NaN,NaN,2020-06-09
657,Mexico,MEXICO,2020-06-09,18993.0,1621.0,15219.0,2153.0,441.0,58.0,563.0,-180.0,NaN,NaN,NaN,NaN,2020-06-09
658,Mexico,CIUDAD DE MEXICO,2020-06-09,31493.0,3804.0,23716.0,3973.0,662.0,75.0,856.0,-269.0,NaN,NaN,NaN,NaN,2020-06-09


In [100]:
pd.read_csv('reporte_Mexico_region_hopkins.csv', sep='|', encoding='UTF-8')

,COUNTRY_REGION,STATE,DATE,CONFIRMED_HM,DEATHS_HM,RECOVERED_HM,ACTIVE_HM,DELTA_CONFIRMED_HM,DELTA_DEATHS_HM,DELTA_RECOVERED_HM,DELTA_ACTIVE_HM,UCI_HM,DELTA_UCI_HM,TESTS_HM,DELTA_TESTS_HM,LAST_UPDATE
0,Mexico,COLIMA,2020-05-21,83.0,10.0,39.0,34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-04
1,Mexico,DURANGO,2020-05-21,189.0,20.0,116.0,53.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-04
2,Mexico,ZACATECAS,2020-05-21,212.0,27.0,134.0,51.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-04
3,Mexico,CAMPECHE,2020-05-21,328.0,48.0,205.0,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-04
4,Mexico,NAYARIT,2020-05-21,349.0,37.0,207.0,105.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,Mexico,TABASCO,2020-06-04,4807.0,592.0,3258.0,957.0,198.0,30.0,198.0,-30.0,NaN,NaN,NaN,NaN,2020-06-04
491,Mexico,BAJA CALIFORNIA,2020-06-04,5460.0,1013.0,3905.0,542.0,229.0,134.0,75.0,20.0,NaN,NaN,NaN,NaN,2020-06-04
492,Mexico,MEXICO,2020-06-04,16320.0,1305.0,13000.0,2015.0,600.0,100.0,415.0,85.0,NaN,NaN,NaN,NaN,2020-06-04
493,Mexico,CIUDAD DE MEXICO,2020-06-04,27326.0,3214.0,20521.0,3591.0,817.0,364.0,771.0,-318.0,NaN,NaN,NaN,NaN,2020-06-04


In [91]:
def check_columns_names(df):
        # Check de que las columnas estén bien.
        columnas = ["COUNTRY_REGION","STATE","DATE","CONFIRMED_HM","DEATHS_HM","RECOVERED_HM",
                    "ACTIVE_HM","DELTA_CONFIRMED_HM","DELTA_DEATHS_HM","DELTA_RECOVERED_HM",
                    "DELTA_ACTIVE_HM","UCI_HM","DELTA_UCI_HM","TESTS_HM","DELTA_TESTS_HM","LAST_UPDATE"]

        filter_ = np.array([True if col in columnas else False for col in df.columns])
        
        if all(filter_):
            print('columns OK')
        else:
            print(f'No match in:\n{df.columns[~filter_x]}')

def check_regions_names(df):
    # Check regiones
    states = ['TOTAL', 'AGUASCALIENTES', 'BAJA CALIFORNIA',
           'BAJA CALIFORNIA SUR', 'CAMPECHE', 'CHIAPAS', 'CHIHUAHUA',
           'CIUDAD DE MEXICO', 'COAHUILA DE ZARAGOZA', 'COLIMA', 'DURANGO',
           'GUANAJUATO', 'GUERRERO', 'HIDALGO', 'JALISCO',
           'MICHOACAN DE OCAMPO', 'MORELOS', 'MEXICO', 'NAYARIT',
           'NUEVO LEON', 'OAXACA', 'PUEBLA', 'QUERETARO', 'QUINTANA ROO',
           'SAN LUIS POTOSI', 'SINALOA', 'SONORA', 'TABASCO', 'TAMAULIPAS',
           'TLAXCALA', 'VERACRUZ DE IGNACIO DE LA LLAVE', 'YUCATAN',
           'ZACATECAS']



    filter_ = np.array([True if x in states else False for x in df.STATE.unique()])

    if all(filter_):
        print('regions OK')
    else:
        print(f'No match in:\n{df.STATE.unique()[~filter_x]}')

In [78]:
path = r'C:\Users\brian\MetLife\Hopkins_region\files' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
#     print(filename)
    df = pd.read_csv(filename)
    if 'Last_Update' in df.columns:
        df['Last_Update'] =  pd.to_datetime(df['Last_Update'])
    elif 'Last Update' in df.columns:
        df['Last Update'] =  pd.to_datetime(df['Last Update'])
        
    li.append(df)

df_daily = pd.concat(li, axis=0, ignore_index=True)
df_daily.loc[pd.isnull(df_daily['Country_Region']),'Country_Region'] = df_daily['Country/Region']
df_daily.loc[pd.isnull(df_daily['Province_State']),'Province_State'] = df_daily['Province/State']
df_daily.loc[pd.isnull(df_daily['Last_Update']),'Last_Update'] = df_daily['Last Update']
df_daily.loc[pd.isnull(df_daily['Lat']),'Lat'] = df_daily['Latitude']
df_daily.loc[pd.isnull(df_daily['Long_']),'Long_'] = df_daily['Longitude']

 

df_daily.drop(columns=['Province/State','Last Update','Latitude','Longitude','FIPS','Admin2','Country/Region','Combined_Key'], inplace=True)
df_daily.sort_values('Last_Update', inplace=True)

df_daily.columns = ['CONFIRMED_HM','DEATHS_HM','RECOVERED_HM','STATE','COUNTRY_REGION','LAST_UPDATE','Lat','Long','ACTIVE_HM', 'Incidence_Rate', 'Case-fatality_Ratio']
df_daily['DATE'] = df_daily['LAST_UPDATE'].dt.date
df_daily['DATE'] =  pd.to_datetime(df_daily['DATE'])

# Fechas primeros registros

In [79]:
''' Primeros registros '''

print(f"Uruguay:\t {df_daily[df_daily['COUNTRY_REGION'] == 'Uruguay']['DATE'].min().date()}")
print(f"Argentina:\t {df_daily[df_daily['COUNTRY_REGION'] == 'Argentina']['DATE'].min().date()}")
print(f"Colombia:\t {df_daily[df_daily['COUNTRY_REGION'] == 'Colombia']['DATE'].min().date()}")
print(f"Chile:\t\t {df_daily[df_daily['COUNTRY_REGION'] == 'Chile']['DATE'].min().date()}")
print(f"Brasil:\t\t {df_daily[df_daily['COUNTRY_REGION'] == 'Brazil']['DATE'].min().date()}")
print(f"Mexico:\t\t {df_daily[df_daily['COUNTRY_REGION'] == 'Mexico']['DATE'].min().date()}")

''' Primeros registros con data por región '''

mex = df_daily[df_daily['COUNTRY_REGION'] == 'Mexico']
print(f"Mexico:\t\t {mex.dropna(subset=['STATE'], axis=0)['DATE'].min().date()}")
col = df_daily[df_daily['COUNTRY_REGION'] == 'Colombia']
print(f"Colombia:\t {col.dropna(subset=['STATE'], axis=0)['DATE'].min().date()}")
chi = df_daily[df_daily['COUNTRY_REGION'] == 'Chile']
print(f"Chile:\t\t {chi.dropna(subset=['STATE'], axis=0)['DATE'].min().date()}")
br = df_daily[df_daily['COUNTRY_REGION'] == 'Brazil']
print(f"Brasil:\t\t {br.dropna(subset=['STATE'], axis=0)['DATE'].min().date()}")

' Primeros registros '

Uruguay:	 2020-03-14
Argentina:	 2020-03-03
Colombia:	 2020-01-23
Chile:		 2020-03-03
Brasil:		 2020-01-23
Mexico:		 2020-01-23


' Primeros registros con data por región '

Mexico:		 2020-05-21
Colombia:	 2020-05-29
Chile:		 2020-05-21
Brasil:		 2020-05-21


In [80]:
first_region_data_date = mex.loc[~pd.isnull(mex['STATE']),'DATE'].min()
aux = mex[mex['DATE'] >= first_region_data_date]

In [81]:
totals_aux = aux.groupby(['DATE']).agg({'DEATHS_HM': 'sum',
                            'ACTIVE_HM': 'sum',
                            'RECOVERED_HM': 'sum',
                            'CONFIRMED_HM': 'sum'}).reset_index()
totals_aux['STATE'] = 'TOTAL'
totals_aux['COUNTRY_REGION'] = 'Mexico'
aux = aux.append(totals_aux).sort_values(by=['DATE', 'CONFIRMED_HM']).reset_index(drop=True)


In [82]:
aux['Fallecidos_corr'] = aux.groupby('STATE')['DEATHS_HM'].transform(lambda x: x.shift())
aux['Activo_corr'] = aux.groupby('STATE')['ACTIVE_HM'].transform(lambda x: x.shift())
aux['Recuperado_corr'] = aux.groupby('STATE')['RECOVERED_HM'].transform(lambda x: x.shift())
aux['Confirmados_corr'] = aux.groupby('STATE')['CONFIRMED_HM'].transform(lambda x: x.shift())

aux['DELTA_CONFIRMED_HM'] = aux['CONFIRMED_HM'] - aux['Confirmados_corr']
aux['DELTA_DEATHS_HM'] = aux['DEATHS_HM'] - aux['Fallecidos_corr']
aux['DELTA_RECOVERED_HM'] = aux['RECOVERED_HM'] - aux['Recuperado_corr']
aux['DELTA_ACTIVE_HM'] = aux['ACTIVE_HM'] - aux['Activo_corr']

aux = aux.drop(['Lat', 'Long', 'Incidence_Rate', 'Case-fatality_Ratio'], axis=1)
aux = aux.drop(['Fallecidos_corr', 'Activo_corr', 'Recuperado_corr', 'Confirmados_corr'], axis=1)

In [83]:
aux = aux.reindex(['COUNTRY_REGION', 'STATE', 'DATE', 'CONFIRMED_HM', 'DEATHS_HM', 'RECOVERED_HM',
            'ACTIVE_HM', 'DELTA_CONFIRMED_HM', 'DELTA_DEATHS_HM', 'DELTA_RECOVERED_HM', 'DELTA_ACTIVE_HM'], axis=1)

aux['UCI_HM'] = np.NaN
aux['DELTA_UCI_HM'] = np.NaN
aux['TESTS_HM'] = np.NaN
aux['DELTA_TESTS_HM'] = np.NaN
aux['LAST_UPDATE'] = aux['DATE'].max()

In [84]:
def replace_tildes(series):
            '''
            Regiones sin tildes y en mayúsculas
            '''
            series = series.str.strip()
            series = series.str.replace('á', 'a')
            series = series.str.replace('é', 'e')
            series = series.str.replace('í', 'i')
            series = series.str.replace('ó', 'o')
            series = series.str.replace('ú', 'u')

            series = series = series.str.upper()
            return series

In [85]:
aux['STATE'] = replace_tildes(aux['STATE'])
aux['STATE'] = aux['STATE'].str.replace("COAHUILA", "COAHUILA DE ZARAGOZA")
aux['STATE'] = aux['STATE'].str.replace("MICHOACAN", "MICHOACAN DE OCAMPO")
aux['STATE'] = aux['STATE'].str.replace("VERACRUZ", "VERACRUZ DE IGNACIO DE LA LLAVE")

In [55]:
aux.to_csv('Mex_region.csv', sep='|', encoding='UTF-8', index=False)

In [89]:
# Check de que las columnas estén bien.
columnas = ["COUNTRY_REGION","STATE","DATE","CONFIRMED_HM","DEATHS_HM","RECOVERED_HM","ACTIVE_HM","DELTA_CONFIRMED_HM","DELTA_DEATHS_HM","DELTA_RECOVERED_HM","DELTA_ACTIVE_HM","UCI_HM","DELTA_UCI_HM","TESTS_HM","DELTA_TESTS_HM","LAST_UPDATE"
]

filter_ = np.array([False if col in columnas else True for col in aux.columns])

aux.columns[filter_]

all(~filter_)

Index([], dtype='object')

True

In [92]:
check_columns_names(aux)

columns OK


In [57]:
# Check states

censo = pd.read_csv(r"C:\Users\brian\MetLife\Censos/Unificado_Censos_Proyecciones_2020.csv", sep=';', encoding='latin9')

filter_mex = censo[censo['COUNTRY_REGION'] == 'Mexico']
states = filter_mex['PROVINCE_STATE'].unique()

aux.STATE.unique()[[False if x in states else True for x in aux.STATE.unique()]]

array([], dtype=object)

In [93]:
check_regions_names(aux)

regions OK


# promedios prueba

In [23]:
# aux.groupby('STATE')['CONFIRMED_HM', 'DEATHS_HM', 'RECOVERED_HM', 'ACTIVE_HM'].sum()
aux['Total_Confirmed_date'] = aux.groupby(['DATE'])['CONFIRMED_HM'].transform('sum')
aux['prom'] = aux['CONFIRMED_HM'] / aux['Total_Confirmed_date']

lista = []
for g, gf in aux.groupby(['DATE'])['STATE', 'prom', 'DATE']:
    lista.append(gf.pivot_table(index='STATE', columns='DATE', margins=False)['prom'].transpose())
    
prom_by_date_by_state = round(pd.concat(lista), 3)

prom_by_date_by_state.index = [x.date() for x in prom_by_date_by_state.index]

prom_by_date_by_state.loc['mean'] = round(prom_by_date_by_state.apply(lambda x: x.mean(), axis=0), 3)
prom_by_date_by_state.loc['std'] = round(prom_by_date_by_state.apply(lambda x: x.std(), axis=0), 5)


prom_by_date_by_state.loc['p'] = prom_by_date_by_state.loc['std'] * 100 / prom_by_date_by_state.loc['mean']

# prom_by_date_by_state.loc['p'].mean()

prom_by_date_by_state

# prom = aux[aux['STATE'] == 'Ciudad de Mexico'][prom']
# prom_ma = prom.rolling(5).mean()
# plt.plot(prom)
# plt.plot(prom_ma)

STATE,Aguascalientes,Baja California,Baja California Sur,Campeche,Chiapas,Chihuahua,Ciudad de Mexico,Coahuila,Colima,Durango,Guanajuato,Guerrero,Hidalgo,Jalisco,Mexico,Michoacan,Morelos,Nayarit,Nuevo Leon,Oaxaca,Puebla,Queretaro,Quintana Roo,San Luis Potosi,Sinaloa,Sonora,TOTAL,Tabasco,Tamaulipas,Tlaxcala,Veracruz,Yucatan,Zacatecas
2020-05-22,0.00500,0.032000,0.00400,0.003,0.008000,0.009000,0.141000,0.006000,0.001,0.002,0.00800,0.00800,0.009000,0.009000,0.081000,0.01000,0.01000,0.003,0.00900,0.00700,0.01500,0.005,0.012000,0.005,0.01900,0.009000,0.5,0.02400,0.009,0.006,0.020000,0.01100,0.002
2020-05-23,0.00500,0.032000,0.00400,0.003,0.008000,0.009000,0.141000,0.007000,0.001,0.002,0.00800,0.00800,0.009000,0.009000,0.081000,0.01000,0.00900,0.003,0.00900,0.00700,0.01500,0.005,0.012000,0.005,0.01900,0.010000,0.5,0.02300,0.009,0.006,0.021000,0.01100,0.002
2020-05-24,0.00500,0.031000,0.00400,0.003,0.008000,0.009000,0.144000,0.006000,0.001,0.002,0.00800,0.00800,0.009000,0.009000,0.081000,0.01000,0.00900,0.003,0.00900,0.00700,0.01500,0.005,0.012000,0.005,0.01900,0.010000,0.5,0.02300,0.009,0.006,0.020000,0.01100,0.002
2020-05-25,0.00400,0.031000,0.00400,0.003,0.008000,0.009000,0.143000,0.006000,0.001,0.002,0.00800,0.00900,0.009000,0.009000,0.081000,0.01000,0.00900,0.003,0.00800,0.00700,0.01500,0.005,0.012000,0.005,0.01900,0.010000,0.5,0.02300,0.009,0.006,0.020000,0.01100,0.002
2020-05-26,0.00500,0.030000,0.00400,0.003,0.009000,0.008000,0.142000,0.006000,0.001,0.002,0.00800,0.00900,0.009000,0.009000,0.081000,0.01000,0.00900,0.003,0.00800,0.00800,0.01500,0.005,0.012000,0.005,0.02000,0.010000,0.5,0.02300,0.009,0.006,0.021000,0.01100,0.002
2020-05-27,0.00500,0.030000,0.00400,0.003,0.009000,0.009000,0.141000,0.006000,0.001,0.002,0.00800,0.00900,0.010000,0.009000,0.083000,0.01000,0.00800,0.003,0.00800,0.00700,0.01600,0.005,0.011000,0.005,0.01900,0.011000,0.5,0.02300,0.009,0.006,0.022000,0.01100,0.002
2020-05-28,0.00500,0.029000,0.00400,0.003,0.009000,0.009000,0.140000,0.006000,0.001,0.002,0.00800,0.00900,0.009000,0.009000,0.084000,0.01000,0.00800,0.003,0.00800,0.00700,0.01600,0.005,0.011000,0.005,0.01900,0.010000,0.5,0.02200,0.009,0.006,0.022000,0.01000,0.002
2020-05-29,0.00500,0.029000,0.00300,0.003,0.009000,0.009000,0.139000,0.006000,0.001,0.002,0.00800,0.01000,0.009000,0.009000,0.084000,0.01100,0.00800,0.003,0.00800,0.00700,0.01600,0.005,0.011000,0.005,0.01900,0.010000,0.5,0.02300,0.009,0.006,0.021000,0.01000,0.002
2020-05-30,0.00500,0.029000,0.00400,0.003,0.009000,0.009000,0.140000,0.006000,0.001,0.002,0.00800,0.01000,0.009000,0.009000,0.083000,0.01100,0.00800,0.003,0.00800,0.00700,0.01700,0.005,0.011000,0.005,0.01800,0.011000,0.5,0.02300,0.009,0.006,0.021000,0.01000,0.002
2020-05-31,0.00500,0.029000,0.00300,0.003,0.010000,0.009000,0.139000,0.006000,0.001,0.002,0.00800,0.01000,0.010000,0.009000,0.082000,0.01100,0.00800,0.003,0.00800,0.00800,0.01700,0.005,0.011000,0.005,0.01900,0.011000,0.5,0.02300,0.009,0.006,0.021000,0.01000,0.002


In [20]:
aux.columns

Index(['COUNTRY_REGION', 'STATE', 'DATE', 'CONFIRMED_HM', 'DEATHS_HM',
       'RECOVERED_HM', 'ACTIVE_HM', 'DELTA_CONFIRMED_HM', 'DELTA_DEATHS_HM',
       'DELTA_RECOVERED_HM', 'DELTA_ACTIVE_HM', 'UCI_HM', 'DELTA_UCI_HM',
       'TESTS_HM', 'DELTA_TESTS_HM', 'LAST_UPDATE', 'Total_Confirmed_date',
       'prom'],
      dtype='object')